In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

# **Functions**

**Bấm để đồng ý cookies**

In [2]:
def accept_cookies(browser):
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Bấm vào các nút > đến hết ở trang discover**

In [3]:
def click_button(browser):
    soup = BeautifulSoup(browser.page_source)
    # lấy tất cả thể loại ở trang discover
    genres = soup.find_all("li", "mixedModularHome__item")

    # chạy lặp hết tất cả thể loại
    for i in range(len(genres)):
        # lấy tất cả playlists của 1 thể loại
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        j = 4   # j: số playlists hiện lên (có thể parse) sau mỗi lần click button
        # lặp đến khi j > số playlists của thể loại đó
        while True:
            if j > len(lists):
                break
            # click vào button
            browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
            j += 4

**Lấy hết tất cả playlists ở trang discover**

In [4]:
def get_all_playlists_discover(browser):
    soup = BeautifulSoup(browser.page_source)
    # lấy tất cả thể loại ở trang discover
    genres = soup.find_all("li", "mixedModularHome__item")

    # chạy lặp hết tất cả thể loại
    for i in range(len(genres)):
        # lấy tất cả playlists của 1 thể loại
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        # trích xuất playlist và url của playlist đó
        for pl in lists:
            title = pl.find("a", "playableTile__heading").get_text().strip()
            user = pl.find("a", "playableTile__usernameHeading")
            link = pl.find("a", "playableTile__artworkLink")["href"]
            # kiểm tra nếu có bị trùng tên
            if title not in playlists:
                playlists["playlist"].append(title)
                if user is None:  
                    playlists["user"].append("Discover")
                else:
                    playlists["user"].append(user.get_text().strip())
                urls_playlist.append("https://soundcloud.com" + link)

**Kéo xuống đến hết trang**

In [5]:
def scroll_down(browser):
    # chạy lặp 2 lần (có thể tăng số lần để đảm bảo scroll hết)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

**Lấy tất cả users và tracks của một playlist**

In [6]:
def get_all_users_tracks_playlist(browser):
    soup = BeautifulSoup(browser.page_source)
    
    # số tracks của 1 playlist
    count = int(soup.find("div", "genericTrackCount__title").get_text())
    
    # isUserPlaylist: điều kiện có phải playlist của user hay không
    # playlist của discover và playlist của user tuy nhìn giống nhau nhưng điều kiện lấy lại khác nhau
    isUserPlaylist = False
    
    # tìm hết tất cả tracks của playlist
    # nếu len(search) bằng 0 (không có) thì playlist này là playlist của user
    search = soup.find_all("li", "systemPlaylistTrackList__item")
    if len(search) == 0:
        isUserPlaylist = True
    
    if isUserPlaylist == False:
        # playlist của discover có thể parse 20 tracks khi không cần scroll
        # nếu lớn hơn sẽ scroll xuống (tiết kiệm thời gian scroll)
        if count > 20:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
        search = soup.find_all("li", "systemPlaylistTrackList__item")
            
    elif isUserPlaylist == True:
        # playlist của user có thể parse 15 tracks khi không cần scroll
        # nếu lớn hơn sẽ scroll xuống (tiết kiệm thời gian scroll)
        if count > 15:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
        search = soup.find_all("li", "trackList__item")

    trackIds = []
    # trích xuất user và track trong 1 playlist
    for producer in search:
        user = producer.find("a", "trackItem__username").get_text()
        url = producer.find("a", "trackItem__username").get('href')
        track = producer.find("a", "trackItem__trackTitle").get_text()
        
        users.add(user)
        urls_user_playlist.add("https://soundcloud.com" + url + "/sets")
        
        # kiểm tra nếu có bị trùng tên
        if track not in tracks["track"]:
            tracks["track"].append(track)
            tracks["user"].append(user)
        trackIds.append(tracks["track"].index(track))
        
    playlists["tracks"].append(trackIds)

**Lấy tất cả playlists của một user**

In [7]:
def get_all_playlists_user(browser):
    soup = BeautifulSoup(browser.page_source)
    posts = soup.find_all("li", "soundList__item")
    
    # kiểm tra có playlist hay không, nếu có chạy dưới, nếu không bỏ qua
    if len(posts) != 0:
        # nếu số playlists lớn hơn 4 thì sẽ scroll
        if len(posts) > 4:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
            
        # trích xuất playlist
        for post in posts:
            # lấy tên playlist
            title = post.find("a", "soundTitle__title").get_text().strip()
            playlists["playlist"].append(title)
            
            # lấy user của playlist
            user = post.find("a", "soundTitle__usernameText").get_text().strip()
            playlists["user"].append(user)

            # lấy Id của các tracks có trong playlist theo index
            songs = post.find_all("li", "compactTrackList__item")
            # lưu index của track
            trackIds = []
            for song in songs:
                trackTitle = song.get_text().strip()
                # nếu có track mới
                if trackTitle not in tracks:
                    tracks["track"].append(trackTitle)
                # lấy index của track
                trackIds.append(tracks["track"].index(trackTitle))
            playlists["user"].append(trackIds)

**Lấy hết tất cả users và tracks của tất cả playlists trích xuất được từ trang discover**

In [8]:
def parse_users_tracks_per_genre(browser):
    for url in urls_playlist:
        # mở tab mới
        browser.execute_script('window.open("{link}");'.format(link = url));
        # chuyển browser về tab mới
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        # thực hiện lấy users và tracks
        get_all_users_tracks_playlist()
        time.sleep(1)
        # đóng tab
        browser.close()
        # chuyển browser về tab discover
        browser.switch_to.window(browser.window_handles[0])

**Lấy tất cả playlists của một user**

In [9]:
def parse_playlists_per_genre(browser):
    for url in urls_user_playlist:
        # mở tab mới
        browser.execute_script('window.open("{link}");'.format(link = url));
        # chuyển browser về tab mới
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        # thực hiện lấy users và tracks
        get_all_playlists_user()
        time.sleep(1)
        # đóng tab
        browser.close()
        # chuyển browser về tab discover
        browser.switch_to.window(browser.window_handles[0])

**Chuyển từ list sang csv/tsv**

In [10]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

________________________________________

# **Main**

**Khởi tạo các list để lưu thông tin**

In [11]:
# Lưu user
users = set()
# Lưu track
tracks = {"track":[], "user":[]}
# Lưu playlist
playlists = {"playlist":[], "user":[], "tracks":[]}
# Lưu link playlist của producer
urls_user_playlist = set()
# Lưu link playlist của discover
urls_playlist = []

In [12]:
browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")

time.sleep(5)
accept_cookies(browser)

time.sleep(1)
click_button(browser)
get_all_playlists_discover(browser)

parse_users_tracks_per_genre(browser)

browser.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@id='onetrust-accept-btn-handler']"}
  (Session info: chrome=95.0.4638.69)
Stacktrace:
0   chromedriver                        0x00000001018d9c99 __gxx_personality_v0 + 579209
1   chromedriver                        0x00000001018655f3 __gxx_personality_v0 + 102371
2   chromedriver                        0x000000010142b288 chromedriver + 172680
3   chromedriver                        0x000000010146025c chromedriver + 389724
4   chromedriver                        0x0000000101492104 chromedriver + 594180
5   chromedriver                        0x000000010147d3cd chromedriver + 508877
6   chromedriver                        0x000000010148feab chromedriver + 585387
7   chromedriver                        0x000000010147d693 chromedriver + 509587
8   chromedriver                        0x0000000101453b7e chromedriver + 338814
9   chromedriver                        0x00000001014

In [ ]:
browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")

time.sleep(5)
accept_cookies(browser)

parse_playlists_per_genre(browser)

browser.quit()

**Lưu data vào file tsv**

In [ ]:
outputFile = "Crawl_data/user.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist.csv"
list_to_tsv_csv(playlists, outputFile)